
# CNN Didática (PyTorch) — Classificação **CIFAR-10** (32×32 RGB)

Este notebook implementa uma **CNN** em **PyTorch** para classificar o **CIFAR-10** (10 classes, imagens 32×32 coloridas).
O foco é **didático**: entender o pipeline **dados → rede → treino → avaliação** e visualizar **filtros** e **feature maps**.

> Obs.: o download do CIFAR-10 ocorre automaticamente (requer internet na primeira execução).


## 1) Imports

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as T
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.mps.is_available() else "cpu")
device



## 2) Dados — CIFAR-10 (download via `torchvision.datasets`)

- Imagens **RGB 32×32**.  
- Normalização com médias e desvios-padrão **CIFAR-10**.  
- *Augmentation* opcional para treino (rotação e flips leves).  


In [ ]:
# Estatísticas padrão do CIFAR-10
CIFAR10_MEAN = (0.4914, 0.4822, 0.4465)
CIFAR10_STD  = (0.2470, 0.2435, 0.2616)

train_tfms = T.Compose([
    T.RandomHorizontalFlip(p=0.5),
    T.RandomCrop(32, padding=4),
    T.ToTensor(),
    T.Normalize(CIFAR10_MEAN, CIFAR10_STD)
])

test_tfms = T.Compose([
    T.ToTensor(),
    T.Normalize(CIFAR10_MEAN, CIFAR10_STD)
])

# Baixa/carrega datasets
train_set = torchvision.datasets.CIFAR10(root="./data", train=True, download=True, transform=train_tfms)
test_set  = torchvision.datasets.CIFAR10(root="./data", train=False, download=True, transform=test_tfms)

train_loader = DataLoader(train_set, batch_size=128, shuffle=True, num_workers=2, pin_memory=True)
test_loader  = DataLoader(test_set,  batch_size=256, shuffle=False, num_workers=2, pin_memory=True)

CLASSES = train_set.classes
len(train_set), len(test_set), CLASSES



## 3) Amostras do dataset (visualização)

Mostramos algumas imagens de treino (desnormalizadas apenas para visualização).


In [ ]:
def unnormalize(img_tensor):
    # img_tensor: (3,H,W) tensor normalized
    mean = torch.tensor(CIFAR10_MEAN).view(3,1,1)
    std  = torch.tensor(CIFAR10_STD).view(3,1,1)
    return img_tensor * std + mean

# Pega um batch
xb, yb = next(iter(train_loader))
xb_vis = xb[:8].cpu()
yb_vis = yb[:8].cpu()

for i in range(xb_vis.size(0)):
    plt.figure(figsize=(3,3))
    img = unnormalize(xb_vis[i]).permute(1,2,0).clamp(0,1).numpy()
    plt.imshow(img)
    plt.title(f"Classe: {CLASSES[yb_vis[i].item()]}")
    plt.axis("off")
    plt.show()



## 4) Modelo — CNN simples para 32×32×3

Arquitetura didática:

- `Conv2d(3→32, k=3, padding=1)` + ReLU  
- `Conv2d(32→32, k=3, padding=1)` + ReLU + `MaxPool2d(2)`  → 32×32 → 16×16  
- `Conv2d(32→64, k=3, padding=1)` + ReLU  
- `Conv2d(64→64, k=3, padding=1)` + ReLU + `MaxPool2d(2)`  → 16×16 → 8×8  
- `Flatten` → `Linear(64×8×8 → 256)` → ReLU → `Linear(256→10)`


In [ ]:
class CifarCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 32->16
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 16->8
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64*8*8, 256),
            nn.ReLU(),
            nn.Linear(256, 10)
        )
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

model = CifarCNN().to(device)
model



## 5) Treino

Usaremos **CrossEntropyLoss** e **Adam**. Reportamos **loss** de treino por época e **acurácia** no conjunto de teste.


In [ ]:
def train(model, train_loader, test_loader, epochs=10, lr=1e-3):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    history = {"train_loss": [], "test_acc": []}

    for epoch in range(1, epochs+1):
        model.train()
        running = 0.0
        for xb, yb in train_loader:
            xb = xb.to(device, non_blocking=True)
            yb = yb.to(device, non_blocking=True)
            logits = model(xb)
            loss = criterion(logits, yb)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running += loss.item() * xb.size(0)

        epoch_loss = running / len(train_loader.dataset)

        # Avaliação
        model.eval()
        all_preds, all_true = [], []
        with torch.no_grad():
            for xb, yb in test_loader:
                xb = xb.to(device, non_blocking=True)
                logits = model(xb)
                preds = torch.argmax(logits, dim=1).cpu().numpy()
                all_preds.append(preds)
                all_true.append(yb.numpy())
        all_preds = np.concatenate(all_preds)
        all_true  = np.concatenate(all_true)
        acc = accuracy_score(all_true, all_preds)

        history["train_loss"].append(epoch_loss)
        history["test_acc"].append(acc)
        print(f"Época {epoch:02d} | Loss treino: {epoch_loss:.4f} | Acurácia teste: {acc:.4f}")
    return history

hist = train(model, train_loader, test_loader, epochs=10, lr=1e-3)



## 6) Curvas (Loss de treino e Acurácia de teste)


In [ ]:
plt.figure(figsize=(7,4))
plt.plot(hist["train_loss"])
plt.xlabel("Época")
plt.ylabel("Loss de treino")
plt.title("Curva de Loss (treino)")
plt.grid(alpha=0.3)
plt.show()

plt.figure(figsize=(7,4))
plt.plot(hist["test_acc"])
plt.xlabel("Época")
plt.ylabel("Acurácia (teste)")
plt.title("Curva de Acurácia (teste)")
plt.grid(alpha=0.3)
plt.show()



## 7) Avaliação final — Matriz de confusão e relatório


In [ ]:
model.eval()
all_preds, all_true = [], []
with torch.no_grad():
    for xb, yb in test_loader:
        xb = xb.to(device, non_blocking=True)
        logits = model(xb)
        preds = torch.argmax(logits, dim=1).cpu().numpy()
        all_preds.append(preds)
        all_true.append(yb.numpy())
all_preds = np.concatenate(all_preds)
all_true  = np.concatenate(all_true)

acc = accuracy_score(all_true, all_preds)
cm  = confusion_matrix(all_true, all_preds)
print(f"Acurácia (teste): {acc:.4f}")
print("Matriz de confusão:\n", cm)
print("\nClassification report:\n", classification_report(all_true, all_preds, target_names=CLASSES))



## 8) Visualizando filtros aprendidos (1ª camada conv — 3×3×3)

Mostramos os **filtros (kernels)** aprendidos na primeira camada (`Conv2d(3→32)`).
Para visualizar, desnormalizamos linearmente cada filtro para [0,1].


In [ ]:
with torch.no_grad():
    conv1_w = model.features[0].weight.cpu().numpy()  # (32,3,3,3)

def minmax01(a):
    a_min, a_max = a.min(), a.max()
    if a_max - a_min < 1e-12:
        return np.zeros_like(a)
    return (a - a_min) / (a_max - a_min)

for i in range(conv1_w.shape[0]):
    w = conv1_w[i]  # (3,3,3)
    w_img = np.transpose(w, (1,2,0))  # (3,3,3) -> HWC
    w_img = minmax01(w_img)
    plt.figure(figsize=(2.5,2.5))
    plt.imshow(w_img)
    plt.title(f"Filtro conv1 #{i}")
    plt.axis("off")
    plt.show()



## 9) Visualizando um feature map (após 1ª conv + ReLU)

Selecionamos uma imagem do **teste**, passamos pela **primeira conv + ReLU** e visualizamos um canal do **mapa de ativação**.


In [ ]:
# Pega uma amostra do teste (desserialize sem augmentation)
xb, yb = next(iter(test_loader))
sample = xb[0:1].to(device)  # (1,3,32,32)

with torch.no_grad():
    conv1 = model.features[0]
    relu  = model.features[1]
    fmap  = relu(conv1(sample))  # (1,32,32,32)

fmap_np = fmap[0].cpu().numpy()  # (32,32,32)

# Mostrar imagem original (desnormalizada)
def unnormalize(img_tensor):
    mean = torch.tensor(CIFAR10_MEAN).view(3,1,1).to(img_tensor.device)
    std  = torch.tensor(CIFAR10_STD ).view(3,1,1).to(img_tensor.device)
    return img_tensor * std + mean

orig = unnormalize(sample[0]).cpu().permute(1,2,0).clamp(0,1).numpy()

plt.figure(figsize=(3.5,3.5))
plt.imshow(orig)
plt.title(f"Original (classe real: {CLASSES[yb[0].item()]})")
plt.axis("off")
plt.show()

plt.figure(figsize=(3.5,3.5))
plt.imshow(fmap_np[0], cmap="viridis")
plt.title("Feature map (canal 0) pós conv1+ReLU")
plt.axis("off")
plt.show()



## 10) Conclusão

- Pipeline completo com **CIFAR-10** (RGB 32×32): carregamento, **CNN**, treino, avaliação, e visualizações.  
- Vimos **filtros aprendidos** (3×3×3) e **feature maps** iniciais — conectando prática e teoria.  
- Extensões possíveis: **data augmentation** mais forte, **regularização** (L2, dropout), e **arquiteturas mais profundas**.
